In [ ]:
import os 
os.chdir("../")

In [ ]:
%pwd

In [1]:
import langchain
import pypdf

from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

print("✅ LangChain berhasil diimport!")
print("✅ PyPDF berhasil diimport!")

✅ LangChain berhasil diimport!
✅ PyPDF berhasil diimport!


In [2]:
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader

def load_pdf_files(data):
    loader = DirectoryLoader(
        data,
        glob="*.pdf",
        loader_cls=PyPDFLoader
    )
    documents = loader.load()
    return documents
    print("data berhasil di load")

In [3]:
extracted_data = load_pdf_files("data")

In [4]:
extracted_data

[Document(metadata={'source': 'data\\data.pdf', 'page': 0}, page_content='Wisata Alam Banyumas \n1. Lokawisata Baturraden  terletak di Karangmangu, Baturraden . Lokawisata \nBaturraden merupakan destinasi utama di kaki Gunung Slamet yang menawarkan \ntaman rekreasi keluarga, kolam air panas, dan area bermain anak. Udara sejuk \npegunungan menjadikannya tempat ideal untuk berlibur bersama keluarga. Harga \ntiket dewasa R p20.000 (weekday), anak -anak Rp10.000 (weekday), dan weekend \nRp25.000 (dewasa), Rp12.500 (anak). Jam buka 7 AM dan jam tutup 7 PM.  \n2. Telaga Sunyi berlokasi di Ketenger, Baturraden. Telaga Sunyi adalah kolam alami \ndengan air jernih di tengah hutan pinus. Suasananya tenang dan sejuk, cocok untuk \nbersantai dan menikmati keindahan alam tanpa keramaian. Harga tiket dewasa \n15000, anak-anak Tidak Ada, weekend Tidak Ada. Jam buka 8 AM dan jam tutup 4 PM. \n3. Hutan Pinus Limpakuwus berlokasi di Jl Baturraden Timur, Limpakuwus, Sumbang. \nHutan Pinus Limpakuwus mena

In [5]:
len (extracted_data)

3

In [6]:
from langchain_core.documents import Document
from typing import List

def filter_to_minimal_docs(docs: List[Document]) -> List[Document]:
    """Simpan hanya page_content dan metadata 'source'."""
    minimal = []
    for d in docs:
        minimal.append(
            Document(
                page_content=d.page_content,
                metadata={"source": d.metadata.get("source")}
            )
        )
    return minimal

# === pakai fungsi & CEK hasilnya ===
min_docs = filter_to_minimal_docs(extracted_data)

# cek 1 dokumen
print(min_docs[0].metadata)          # -> {'source': 'data\\data.pdf'}
# pastikan tidak ada kunci lain:
print(list(min_docs[0].metadata.keys()))



{'source': 'data\\data.pdf'}
['source']


In [7]:
minimal_docs = filter_to_minimal_docs(extracted_data)

In [8]:
extracted_data

[Document(metadata={'source': 'data\\data.pdf', 'page': 0}, page_content='Wisata Alam Banyumas \n1. Lokawisata Baturraden  terletak di Karangmangu, Baturraden . Lokawisata \nBaturraden merupakan destinasi utama di kaki Gunung Slamet yang menawarkan \ntaman rekreasi keluarga, kolam air panas, dan area bermain anak. Udara sejuk \npegunungan menjadikannya tempat ideal untuk berlibur bersama keluarga. Harga \ntiket dewasa R p20.000 (weekday), anak -anak Rp10.000 (weekday), dan weekend \nRp25.000 (dewasa), Rp12.500 (anak). Jam buka 7 AM dan jam tutup 7 PM.  \n2. Telaga Sunyi berlokasi di Ketenger, Baturraden. Telaga Sunyi adalah kolam alami \ndengan air jernih di tengah hutan pinus. Suasananya tenang dan sejuk, cocok untuk \nbersantai dan menikmati keindahan alam tanpa keramaian. Harga tiket dewasa \n15000, anak-anak Tidak Ada, weekend Tidak Ada. Jam buka 8 AM dan jam tutup 4 PM. \n3. Hutan Pinus Limpakuwus berlokasi di Jl Baturraden Timur, Limpakuwus, Sumbang. \nHutan Pinus Limpakuwus mena

In [9]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
def text_split(minimal_docs):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=20,
    )
    texts_chunk = text_splitter.split_documents(minimal_docs)
    return texts_chunk

In [10]:
texts_chunk = text_split(minimal_docs)
print(f"Number of chunks: {len(texts_chunk)}")

Number of chunks: 20


In [11]:
texts_chunk

[Document(metadata={'source': 'data\\data.pdf'}, page_content='Wisata Alam Banyumas \n1. Lokawisata Baturraden  terletak di Karangmangu, Baturraden . Lokawisata \nBaturraden merupakan destinasi utama di kaki Gunung Slamet yang menawarkan \ntaman rekreasi keluarga, kolam air panas, dan area bermain anak. Udara sejuk \npegunungan menjadikannya tempat ideal untuk berlibur bersama keluarga. Harga \ntiket dewasa R p20.000 (weekday), anak -anak Rp10.000 (weekday), dan weekend \nRp25.000 (dewasa), Rp12.500 (anak). Jam buka 7 AM dan jam tutup 7 PM.'),
 Document(metadata={'source': 'data\\data.pdf'}, page_content='2. Telaga Sunyi berlokasi di Ketenger, Baturraden. Telaga Sunyi adalah kolam alami \ndengan air jernih di tengah hutan pinus. Suasananya tenang dan sejuk, cocok untuk \nbersantai dan menikmati keindahan alam tanpa keramaian. Harga tiket dewasa \n15000, anak-anak Tidak Ada, weekend Tidak Ada. Jam buka 8 AM dan jam tutup 4 PM. \n3. Hutan Pinus Limpakuwus berlokasi di Jl Baturraden Timur

In [12]:
from langchain_huggingface import HuggingFaceEmbeddings

def download_embeddings():
    model_name = "sentence-transformers/all-MiniLM-L6-v2"
    return HuggingFaceEmbeddings(
        model_name=model_name,
        model_kwargs={"device": "cpu"},
        encode_kwargs={"normalize_embeddings": True},
    )

embeddings = download_embeddings()
print("OK:", len(embeddings.embed_query("tes")))

c:\Users\akuns\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


OK: 384


In [13]:
embeddings

HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={'device': 'cpu'}, encode_kwargs={'normalize_embeddings': True}, query_encode_kwargs={}, multi_process=False, show_progress=False)

In [14]:
daftar_teks = ["Hallo dunia"]
vectors = embeddings.embed_documents(daftar_teks)

print(vectors)

[[-0.03803442791104317, 0.004518091678619385, -0.060600556433200836, -0.011571232229471207, -0.10467272251844406, -0.02041025459766388, 0.11283578723669052, -0.06815135478973389, -0.021047642454504967, 0.043138157576322556, -0.0011391652515158057, -0.03410258889198303, -0.05171287804841995, -0.024494566023349762, -0.03319265693426132, 0.06849576532840729, 0.022722933441400528, 0.12076646089553833, -0.06955042481422424, -0.03025602363049984, 0.07576902955770493, -0.027407191693782806, -0.04367559775710106, 0.08945557475090027, -0.055883366614580154, 0.017747117206454277, 0.09197574108839035, 0.060666605830192566, -0.08802050352096558, -0.10222357511520386, -0.07262267172336578, 0.1297530084848404, 0.021427178755402565, -0.023860635235905647, -0.022033073008060455, 0.01222624909132719, -0.02029860019683838, -0.011958613991737366, 0.051862891763448715, -0.017058730125427246, 0.00247299880720675, -0.05793122202157974, 0.04660619795322418, -0.026811793446540833, 0.025415461510419846, -0.036

In [15]:
print(f"Vector length: {len(vectors[0])}")

Vector length: 384


In [16]:
from dotenv import load_dotenv
import os
load_dotenv()

True

#LOAD API

In [17]:
from dotenv import load_dotenv
import os

load_dotenv()

PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

if not PINECONE_API_KEY or not OPENAI_API_KEY:
    print("Error: Pastikan API key sudah diatur di file .env")
else:
    print("API keys berhasil dimuat!")

API keys berhasil dimuat!


In [18]:
import pinecone
print("Pinecone version:", pinecone.__version__)


Pinecone version: 7.3.0


In [19]:
from pinecone import Pinecone, ServerlessSpec
pinecone_api_key = PINECONE_API_KEY

pc= Pinecone(api_key=pinecone_api_key)

In [20]:
pc

In [21]:
from pinecone import ServerlessSpec 

index_name = "compare-models-chatbot"

if not pc.has_index(index_name):
    pc.create_index(
        name = index_name,
        dimension=384,  
        metric= "cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )


index = pc.Index(index_name)

In [22]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=texts_chunk,
    embedding=embeddings,
    index_name=index_name
)

c:\Users\akuns\AppData\Local\Programs\Python\Python312\Lib\site-packages\langchain_pinecone\__init__.py:3: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain_pinecone.vectorstores import Pinecone, PineconeVectorStore


In [23]:
from langchain_pinecone import PineconeVectorStore
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [24]:
dswitch = Document(
    page_content="Taman Panemon yang berlokasi di Kebanggan, Sumbang, Banyumas merupakan destinasi wisata keluarga dengan konsep kolam renang bernuansa pantai, di mana bagian tepi kolam lebih dangkal dan di tengahnya terdapat pulau kecil dengan pohon kelapa. Suasananya semakin menarik dengan taman bunga warna-warni yang indah dan area yang cocok untuk berfoto. Pengunjung juga dapat menikmati aktivitas seru seperti sewa mobil offroad untuk berkeliling area wisata. Fasilitas umum seperti toilet yang bersih turut menambah kenyamanan pengunjung. Harga tiket masuk sebesar Rp15.000 untuk dewasa, Rp20.000 saat akhir pekan, dan jam operasionalnya mulai pukul 08.00 hingga 17.00.",
    metadata={"source": "website"}
)

In [25]:
docsearch.add_documents(documents=[dswitch])

['a5142657-af61-4419-b0fe-f13280bf9269']

In [26]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [27]:
from langchain_core.globals import set_debug
set_debug(True)
retrieved_docs = retriever.invoke("harga tiket pancuran pitu ?")
retrieved_docs

[Document(id='dcc686d7-285c-4d36-8d76-a39c57ba642f', metadata={'source': 'data\\data.pdf'}, page_content='dipercaya bermanfaat untuk relaksasi dan kesehatan kulit. Harga tiket dewasa \nRp20.000 (wkdy), anak -anak Tidak Ada, weekend Rp25.000 (wkd). Jam buka 7 A M \ndan jam tutup 5 PM. \n15. Pancuran 7 (Pitu)  berada di Ketenger, Baturraden. Pancuran Pitu dikenal sebagai \npemandian air panas dengan tujuh pancuran alami. Pengunjung bisa merendam \ntubuh sambil menikmati udara pegunungan yang sejuk. Harga tiket dewasa 15000, \nanak-anak Tidak Ada, weekend Tidak Ada. Jam buka 8 AM dan jam tutup 4 PM .'),
 Document(id='192f873b-fee1-4d15-b33a-4e0586290651', metadata={'source': 'data\\data.pdf'}, page_content='dipercaya bermanfaat untuk relaksasi dan kesehatan kulit. Harga tiket dewasa \nRp20.000 (wkdy), anak -anak Tidak Ada, weekend Rp25.000 (wkd). Jam buka 7 A M \ndan jam tutup 5 PM. \n15. Pancuran 7 (Pitu)  berada di Ketenger, Baturraden. Pancuran Pitu dikenal sebagai \npemandian air pana

In [28]:
retrieved_docs = retriever.invoke("harga tiket anak-anak pancuran pitu ?")
retrieved_docs

[Document(id='dcc686d7-285c-4d36-8d76-a39c57ba642f', metadata={'source': 'data\\data.pdf'}, page_content='dipercaya bermanfaat untuk relaksasi dan kesehatan kulit. Harga tiket dewasa \nRp20.000 (wkdy), anak -anak Tidak Ada, weekend Rp25.000 (wkd). Jam buka 7 A M \ndan jam tutup 5 PM. \n15. Pancuran 7 (Pitu)  berada di Ketenger, Baturraden. Pancuran Pitu dikenal sebagai \npemandian air panas dengan tujuh pancuran alami. Pengunjung bisa merendam \ntubuh sambil menikmati udara pegunungan yang sejuk. Harga tiket dewasa 15000, \nanak-anak Tidak Ada, weekend Tidak Ada. Jam buka 8 AM dan jam tutup 4 PM .'),
 Document(id='2a3eaa19-7cbe-4017-bb63-fe62f7b244d7', metadata={'source': 'data\\data.pdf'}, page_content='dipercaya bermanfaat untuk relaksasi dan kesehatan kulit. Harga tiket dewasa \nRp20.000 (wkdy), anak -anak Tidak Ada, weekend Rp25.000 (wkd). Jam buka 7 A M \ndan jam tutup 5 PM. \n15. Pancuran 7 (Pitu)  berada di Ketenger, Baturraden. Pancuran Pitu dikenal sebagai \npemandian air pana

In [29]:
# Initialize client with SumoPod AI
from langchain_core.globals import set_verbose
set_verbose(True)

from langchain_openai import ChatOpenAI

chatModel = ChatOpenAI(model="gpt-4o-mini")


#CEK DULU LONGCHAINNYA

In [30]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [31]:
system_prompt = (
    "You are an Tourism assistant for question-answering tasks use indonesia language. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [32]:
question_answer_chain = create_stuff_documents_chain(chatModel, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [35]:
response = rag_chain.invoke({"input": "berapa harga tiket pancuran pitu untuk anak-anak?"})
print(response["answer"])

[chain/start] [chain:retrieval_chain] Entering Chain run with input:
{
  "input": "berapa harga tiket pancuran pitu untuk anak-anak?"
}
[chain/start] [chain:retrieval_chain > chain:RunnableAssign<context>] Entering Chain run with input:
{
  "input": "berapa harga tiket pancuran pitu untuk anak-anak?"
}
[chain/start] [chain:retrieval_chain > chain:RunnableAssign<context> > chain:RunnableParallel<context>] Entering Chain run with input:
{
  "input": "berapa harga tiket pancuran pitu untuk anak-anak?"
}
[chain/start] [chain:retrieval_chain > chain:RunnableAssign<context> > chain:RunnableParallel<context> > chain:retrieve_documents] Entering Chain run with input:
{
  "input": "berapa harga tiket pancuran pitu untuk anak-anak?"
}
[chain/start] [chain:retrieval_chain > chain:RunnableAssign<context> > chain:RunnableParallel<context> > chain:retrieve_documents > chain:RunnableLambda] Entering Chain run with input:
{
  "input": "berapa harga tiket pancuran pitu untuk anak-anak?"
}
[chain/end] [

In [37]:
response = rag_chain.invoke({"input": "rekomendasi wisata alam di banyumas ?"})
print(response["answer"])

[chain/start] [chain:retrieval_chain] Entering Chain run with input:
{
  "input": "rekomendasi wisata alam di banyumas ?"
}
[chain/start] [chain:retrieval_chain > chain:RunnableAssign<context>] Entering Chain run with input:
{
  "input": "rekomendasi wisata alam di banyumas ?"
}
[chain/start] [chain:retrieval_chain > chain:RunnableAssign<context> > chain:RunnableParallel<context>] Entering Chain run with input:
{
  "input": "rekomendasi wisata alam di banyumas ?"
}
[chain/start] [chain:retrieval_chain > chain:RunnableAssign<context> > chain:RunnableParallel<context> > chain:retrieve_documents] Entering Chain run with input:
{
  "input": "rekomendasi wisata alam di banyumas ?"
}
[chain/start] [chain:retrieval_chain > chain:RunnableAssign<context> > chain:RunnableParallel<context> > chain:retrieve_documents > chain:RunnableLambda] Entering Chain run with input:
{
  "input": "rekomendasi wisata alam di banyumas ?"
}
[chain/end] [chain:retrieval_chain > chain:RunnableAssign<context> > chai

In [39]:
response = rag_chain.invoke({"input": " rekomendasi wisata di banyumas selain wisata alam?"})
print(response["answer"])


[chain/start] [chain:retrieval_chain] Entering Chain run with input:
{
  "input": " rekomendasi wisata di banyumas selain wisata alam?"
}
[chain/start] [chain:retrieval_chain > chain:RunnableAssign<context>] Entering Chain run with input:
{
  "input": " rekomendasi wisata di banyumas selain wisata alam?"
}
[chain/start] [chain:retrieval_chain > chain:RunnableAssign<context> > chain:RunnableParallel<context>] Entering Chain run with input:
{
  "input": " rekomendasi wisata di banyumas selain wisata alam?"
}
[chain/start] [chain:retrieval_chain > chain:RunnableAssign<context> > chain:RunnableParallel<context> > chain:retrieve_documents] Entering Chain run with input:
{
  "input": " rekomendasi wisata di banyumas selain wisata alam?"
}
[chain/start] [chain:retrieval_chain > chain:RunnableAssign<context> > chain:RunnableParallel<context> > chain:retrieve_documents > chain:RunnableLambda] Entering Chain run with input:
{
  "input": " rekomendasi wisata di banyumas selain wisata alam?"
}
[ch